# Literary Application

In [3]:
%load_ext literary.notebook

In [4]:
from pathlib import Path

from traitlets import Unicode
from traitlets.config import Application, catch_config_error

from ..core.config import find_config_file

In [3]:
class LiteraryApp(Application):
    name = "literary"
    description = "A Literary application"
    
    def load_app_config_file():
        file_path = find_config_file(Path.cwd())
        self.load_config_file(file_path.name, str(file_path))
    
    @catch_config_error
    def initialize(self, argv=None):
        self.load_app_config_file()
        self.parse_command_line(argv)